# Семинар №8
## Классы (часть 2)
  
  
Сегодня в меню:
  * базовые магические методы
  * магические методы для арифметики
  * магические методы для приведения типов
  * итерирование
  * динамическая работа с аттрибутами
  * ```__getattr__``` vs. ```__getattribute__```
  * контексты
  * создание и удаление объектов

### базовые магические методы

In [1]:
import random

class Vector:        
    def __init__(self, x=0, y=0, color=None):
        print("initializing a vector")
        if type(x) != int or type(y) != int:
            raise AttributeError('x and y should be int')
        
        self._x = x
        self._y = y
        self._color = color
    
    def get_x(self):
        return self._x
    
    def get_y(self):
        return self._y

Методы с двойным подчеркиванием в начале и конце имени имеют особое значение. 

Мы уже знакомы с `__next__` и `__iter__`, пора узнать и об остальных

In [2]:
vector = Vector(1, 2, 'red')
str(vector)

initializing a vector


'<__main__.Vector object at 0x7f0a68031c70>'

In [3]:
class VectorWithStr(Vector):
    def __str__(self):
        return 'vector ({}, {}) of color {}'.format(self._x, self._y, self._color)

In [4]:
vector = VectorWithStr(1, 2, 'red')
str(vector)

initializing a vector


'vector (1, 2) of color red'

__Q:__ Преобразование в строку? Это всё?

__A:__ Конечно, нет. Неявные преобразования иногда происходят там, где мы их не ожидаем

In [5]:
print(vector)

vector (1, 2) of color red


In [8]:
mylist = [vector]
print(mylist)

__Q:__ А откуда опять "некрасивые" строки?!


__A:__ В питоне используется два способа приведения к строке. Это функции `str` и `repr`, которые отличаются своим назначением. 

`str` используется там, где нужна человекочитаемость, а `repr` реализуется так, чтобы можно было однозначно определить, о каком объекте идет речь. Если `repr` не реализован, используется стандартный вариант, а если не реализован `str`, то вместо него используется `repr`. 

Попробуем?

In [9]:
class VectorWithRepr(Vector):
    def __repr__(self):
        return 'vector representation (x: {}, y: {}, color: {})'.format(self._x, self._y, self._color)

In [11]:
vector = VectorWithRepr(1, 2, 'red')

print(vector)
mylist = [vector]
print(mylist)

initializing a vector
vector representation (x: 1, y: 2, color: red)
[vector representation (x: 1, y: 2, color: red)]


In [12]:
class VectorWithBothReprAndStr(VectorWithRepr, VectorWithStr):
    pass

In [13]:
vector = VectorWithBothReprAndStr(1, 2, 'red')
# вот здесь должны получиться разные значения
print(vector)
print([vector])

initializing a vector
vector (1, 2) of color red
[vector representation (x: 1, y: 2, color: red)]


### магические методы для арифметики

In [14]:
import math
import random

class VectorWithMath(VectorWithBothReprAndStr):    
    def __abs__(self):
        return math.hypot(self._x, self._y)
    
    def __add__(self, other):
        return VectorWithMath(self.get_x() + other.get_x(),
                     self.get_y() + other.get_y(),
                     random.choice((str(self._color), str(other._color))))
    
    def __sub__(self, other):
        return VectorWithMath(self.get_x() - other.get_x(),
                     self.get_y() - other.get_y(),
                     random.choice((str(self._color), str(other._color))))
    
    # ещё есть div, mul и многое другое

In [15]:
vector1 = VectorWithMath(3, 4, 'blue')
vector2 = VectorWithMath(1, 2, 'red')
print(vector1)
print(vector2)

initializing a vector
initializing a vector
vector (3, 4) of color blue
vector (1, 2) of color red


In [16]:
print(abs(vector1))
print(vector1 + vector2)
print(vector1 - vector2)

5.0
initializing a vector
vector (4, 6) of color red
initializing a vector
vector (2, 2) of color red


### магические методы для приведения типов

In [17]:
import math

class VectorWithTypes(VectorWithMath):
    def __bool__(self):
        return bool(self._x) or bool(self._y)
    
    def __int__(self):
        return int(float(self))
    
    def __float__(self):
        return abs(self)

In [18]:
vector = VectorWithTypes(3, 4, 'blue')
print(vector)
print(int(vector))
print(float(vector))

print("vector ~ True") if vector else print("vector ~ False")

initializing a vector
vector (3, 4) of color blue
5
5.0
vector ~ True


In [19]:
vector = VectorWithTypes()
print(vector)

print("vector ~ True") if vector else print("vector ~ False")

initializing a vector
vector (0, 0) of color None
vector ~ False


### итерирование

Один способ сделать объект "итерабельным" нам уже известен, это метод `__next__`. Но он не единственный

In [20]:
class VectorIterable(VectorWithTypes):
    def __getitem__(self, position):
        return (self._x, self._y)[position]
    
    def __len__(self):
        return 2
    
    def __reversed__(self):
        return (self._x, self._y)[::-1]

In [21]:
vector = VectorIterable(100, 500)
print(vector[0])
print(vector[3])

initializing a vector
100


IndexError: tuple index out of range

In [22]:
# for вызывает __getitem__ ТОЛЬКО если у класса отсутствует __iter__ 

for coordinate in vector:
    print(coordinate)

100
500


In [23]:
for coordinate in reversed(vector):
    print(coordinate)

500
100


In [24]:
class VectorIterable(VectorWithTypes):
    
    def __getitem__(self, position):
        return (self._y, self._x)[position]
    
    def __iter__(self):
        
        #return iter((self._x, self._y))
        return self
    
    def __next__(self):
        
        if not hasattr(self,'count'):
            self.count = 0
        
        self.count += 1
        
        if self.count < 3:
            return (self._x, self._y)[self.count-1]
        else:
            raise StopIteration
    
    def __len__(self):
        return 2
    
    def __reversed__(self):
        return (self._x, self._y)[::-1]

In [25]:
vect = VectorIterable(3, 5)

for c in vect:
    print(c)

initializing a vector
3
5


### динамическая работа с атрибутами

Казалось бы, в питоне нет никакой защиты от "взлома". Но нельзя ли сделать её самостоятельно?

In [26]:
class VectorWithAllAttributes(VectorIterable):
    def __getattr__(self, attr_name):
        return "value of {}".format(attr_name)
    
    def __setattr__(self, attr_name, attr_value):
        if attr_name not in ('_x', '_y', '_color'):
            raise Exception('you shall not add new attributes here, young padawan!')
        else:
            super().__setattr__(attr_name, attr_value)
            
    def __delattr__(self, attr_name):
        print('Heh, you can delete nothing')

In [27]:
vector = VectorWithAllAttributes(1, 2, 'violet')
print(dir(vector))

initializing a vector
['__abs__', '__add__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__float__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__int__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_color', '_x', '_y', 'get_x', 'get_y']


In [28]:
print(vector.some_attribute)
print(vector._color)
print(vector.get_x())

value of some_attribute
violet
1


In [29]:
vector.new_attribute = "value"

Exception: you shall not add new attributes here, young padawan!

In [30]:
del vector._color
delattr(vector, '_color')
print(vector._color)

Heh, you can delete nothing
Heh, you can delete nothing
violet


### ```__getattr__ vs. __getattribute__```

In [31]:
class GetAttr:
    attr1 = 1
    def __init__(self):
        self.attr2 = 2
    def __getattr__(self, attr):   # Только для неопределенных атрибутов
        print('get: ' + attr)      # Не attr1: наследуется от класса
        return 3                   # Не attr2: хранится в экземпляре
    

class GetAttribute:
    attr1 = 1
    def __init__(self):
        self.attr2 = 2
    def __getattribute__(self, attr):  # Вызывается всеми операциями присваивания
        print('get: ' + attr)          # Для предотвращения зацикливания используется суперкласс
        if attr == 'attr3':
            return 3
        else:
            return super().__getattribute__(attr)


In [32]:
X = GetAttr()
print(X.attr1)
print(X.attr2)
print(X.attr3)
print('-' * 40)        
X = GetAttribute()
print(X.attr1)
print(X.attr2)
print(X.attr3)

1
2
get: attr3
3
----------------------------------------
get: attr1
1
get: attr2
2
get: attr3
3


### контексты

In [33]:
class VectorWithContextManager(VectorWithAllAttributes):
    def __enter__(self):
        print('entering context')
    def __exit__(self, *args):
        print(args)
        print(dir(args[2]),args[2].tb_lineno)
        print('leaving context')
        #return False # -- бросаем ошибку дальше
        return True  # -- НЕ бросаем ошибку дальше

In [34]:
try:
    with VectorWithContextManager() as vec:
        for i in range(3):
            print(i)
        raise Exception('something happened inside!')
except:
    print('an exception was raised...')
    pass
print('we are out of the context')

initializing a vector
entering context
0
1
2
(<class 'Exception'>, Exception('something happened inside!'), <traceback object at 0x7f0a5a703140>)
['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next'] 5
leaving context
we are out of the context


Но можно и проще!

In [35]:
from contextlib import contextmanager

@contextmanager
def vector_mgr():
    print('handling entering the context')
    yield Vector()
    print('handling leaving the context')
          
print('statement before context')
with vector_mgr() as vector:
    for i in range(3):
        print(vector)
print('statement after context')

statement before context
handling entering the context
initializing a vector
handling leaving the context
statement after context


In [36]:
# А теперь с ошибкой:

@contextmanager
def vector_mgr():
    try:
        print('handling entering the context')
        yield Vector()
    except ZeroDivisionError as e:
        pass
    finally:
        print('handling leaving the context')
          
print('statement before context')
with vector_mgr() as vector:
    for i in range(3):
        print(vector)
        infin = 1 / 0
print('statement after context')

statement before context
handling entering the context
initializing a vector
handling leaving the context
statement after context


### cоздание и удаление объектов

In [37]:
class VectorInitialized(VectorWithContextManager):
    def __new__(cls, *args, **kwargs):
        print('invoking __new__ method')
        print(cls, args, kwargs)
        print(object)
        return object.__new__(cls)
    
    def __del__(self):
        print('deleting an object')
        raise Exception("exception while destructing")

In [38]:
vect = VectorInitialized(1, 2, color='navy blue')
print(vect)

invoking __new__ method
<class '__main__.VectorInitialized'> (1, 2) {'color': 'navy blue'}
<class 'object'>
initializing a vector
vector (1, 2) of color navy blue


In [39]:
del vect

Exception ignored in: <function VectorInitialized.__del__ at 0x7f0a5a77baf0>
Traceback (most recent call last):
  File "<ipython-input-37-0767888d90a6>", line 10, in __del__
Exception: exception while destructing


deleting an object
